In [ ]:
from models import *

import torch_geometric.transforms as T
from torch_geometric.datasets import ModelNet
from torch_geometric.loader import DataLoader

# Hyperparameters
EPOCHS = 100

train_dataset = ModelNet('./dataset', name='10', transform=T.SamplePoints(1000))
test_dataset = ModelNet('./dataset', name='10', train=False, transform=T.SamplePoints(1000))
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)
pointnet = PointNet(3, 10).to(device)

In [ ]:
losses = []
best_validation = 0
best_model = None
for e in range(EPOCHS):
    print(f" epoch {e+1}/{EPOCHS}")
    loss, predict, truth, v_predict, v_truth = train(pointnet, train_loader, test_loader)
    matches = 0
    for i, _ in enumerate(truth):
        if predict[i]==truth[i]: matches += 1
    v_matches = 0
    for i, _ in enumerate(v_truth):
        if v_predict[i]==v_truth[i]: v_matches += 1
    v_acc = v_matches/len(v_truth)
    print(f" loss = {loss}", "Accuracy Training: {} - Validation: {}".format(matches/len(truth), v_acc))
    losses.append(loss)
    if best_validation < v_acc: 
        torch.save(pointnet.state_dict(), 'saved_best_model.pkl')
        best_validation = v_acc

In [ ]:
data = next(iter(test_loader))

In [ ]:
index = 3121

# pos = (data.pos[:2500] - torch.mean(data.pos[:2500]))/torch.std(data.pos[:2500])
pos = data.pos[:2500] 
pos = shift_point_cloud(pos.unsqueeze(0))
x = pos[:,:,0]
y = pos[:,:,1]
z = pos[:,:,2]

%matplotlib widget
  
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
# creating figure
fig = plt.figure()
ax = Axes3D(fig)
# creating the plot
plot_geeks = ax.scatter(x, y, z, color='green')
# setting title and labels
ax.set_title("3D plot")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
# displaying the plot
plt.show()